In [1]:
pip install BeautifulSoup4 openpyxl unidecode scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 9.3 MB/s eta 0:00:00


In [6]:
!wget https://datos.cdmx.gob.mx/dataset/75fda961-2c64-4671-bdef-a61feb1ec1cb/resource/12d71b2e-1ae0-44e8-8ee8-7c5e3489adde/download/grado-de-marginalidad-y-violencia-urbana-por-colonia-en-la-ciudad-de-mxico.json

--2024-09-14 05:35:41--  https://datos.cdmx.gob.mx/dataset/75fda961-2c64-4671-bdef-a61feb1ec1cb/resource/12d71b2e-1ae0-44e8-8ee8-7c5e3489adde/download/grado-de-marginalidad-y-violencia-urbana-por-colonia-en-la-ciudad-de-mxico.json
Resolving datos.cdmx.gob.mx (datos.cdmx.gob.mx)... 189.240.234.183
Connecting to datos.cdmx.gob.mx (datos.cdmx.gob.mx)|189.240.234.183|:443... ^C


In [3]:
import requests
from bs4 import BeautifulSoup as bs
import json
import pandas as pd
import openpyxl as xls
from unidecode import unidecode
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
## Leemos el archivo de grado de marginalidad y violencia urbana de CDMX y obtenemos solo los datos que nos interesan
## El grado de marginalidad y la colonia
marg_x_cols = []
with open('grado-de-marginalidad-y-violencia-urbana-por-colonia-en-la-ciudad-de-mxico.json','r') as f:
    grado_marginalidad = json.load(f)
    #print(grado_marginalidad
    marg_x_cols = [[i['properties']['alcaldia'],i['properties']['cve_col'],
                    i['properties']['colonia'],i['properties']['pob_2010'],
                    i['properties']['C_US']]  for i in grado_marginalidad['features']]
    del grado_marginalidad
marg_x_cols_df = pd.DataFrame(marg_x_cols,columns=['alcaldia','cve_col','colonia','pob_2010','grado_marginalidad'])
marg_x_cols_df.head()

,alcaldia,cve_col,colonia,pob_2010,grado_marginalidad
0,AZCAPOTZALCO,02-001,AGUILERA,2014.0,4
1,AZCAPOTZALCO,02-002,ALDANA,3378.0,5
2,AZCAPOTZALCO,02-005,ANGEL ZIMBRON,2737.0,2
3,AZCAPOTZALCO,02-006,ARENAL,4817.0,4
4,AZCAPOTZALCO,02-007,CENTRO DE AZCAPOTZALCO,3043.0,1


In [15]:
## Leemos los codigos postales obtenidos por parte de correos de mexico
## https://www.portal.correosdemexico.com.mx/portal/index.php/envio/consulta-de-codigo-postal
## Y les damos formato que podamos cruzar contra el grado marginal
codigos_postales = pd.read_csv('cat_colonias_ine.csv',header=2)
codigos_postales.drop(columns=['Unnamed: 0'],inplace=True)
codigos_postales.rename(columns={'TIPO DE SECCION':'tipo_seccion',
                                 'TIPO DE COLONIA':'tipo_col',
                                 'NOMBRE DE LA COLONIA':'nombre_col',
                                 'NOMBRE DEL MUNICIPIO':'municipio'
                                 },inplace=True)
codigos_postales.drop_duplicates(['nombre_col','CP'],inplace=True)
codigos_postales = codigos_postales[['tipo_seccion','tipo_col','nombre_col','CP','municipio']]
codigos_postales.head()

,tipo_seccion,tipo_col,nombre_col,CP,municipio
0,URBANO(A),AMPLIACION,MALACATES,7119,GUSTAVO A. MADERO
4,URBANO(A),COLONIA,VERONICA CASTRO,7140,GUSTAVO A. MADERO
8,URBANO(A),COLONIA,ARBOLEDAS DE CUAUTEPEC,7140,GUSTAVO A. MADERO
10,URBANO(A),COLONIA,LA FORESTAL 1,7140,GUSTAVO A. MADERO
11,URBANO(A),COLONIA,LA FORESTAL 2,7144,GUSTAVO A. MADERO


In [26]:
# Basados en la ayuda de gemini de google y copilot encontramos que una manera confiable de comparar dos
# cadenas de texto podia ser mediante la similitud coseno que podemos implementar desde sklearn
def str_compare(str1, str2):
    def normalize_str(strAux):
        str_out = re.sub(r'[^a-zA-Z|^\s]', '', strAux)
        return str_out.lower()
    str1_norm = normalize_str(str1)
    str2_norm = normalize_str(str2)
    # Utilizamos la similitud coseno para comaprar cadenas
    vectorizer = CountVectorizer().fit_transform([str1_norm, str2_norm])
    vectors = vectorizer.toarray()
    cos_sim = cosine_similarity(vectors)
    umbral = 0.70  # 70% de similitud
    if cos_sim[0][1] >= umbral:
        print(str1_norm)
        print(str2_norm)
        print(cos_sim[0][1]) # Devuelve la similitud coseno
        return True
    else:
        return False

In [27]:
str_compare('ARBOLEDAS DE CUAUTEPEC','VERONICA CASTRO')

False

In [28]:
# agregamos la informacion del dataframe codigos_postales al df marg_x_cols_df tomando en cuenta la condicion de que el
# campo colonia se encuentre dentro de la columna nombre_col utilizando para comparar las cadenas la funcion str_compare

marg_x_cols_df['CP'] = ''
for index, row in marg_x_cols_df.iterrows():
  for index2, row2 in codigos_postales[codigos_postales['municipio']==row['alcaldia']].iterrows():
    if str_compare(row['colonia'],row2['nombre_col']):
      marg_x_cols_df.loc[index, 'CP'] = row2['CP']
      break


aguilera
aguilera
1.0
aldana
aldana
1.0
angel zimbron
angel zimbron
0.9999999999999998
arenal
arenal
1.0
centro de azcapotzalco
centro de azcapotzalco col villa azcapotzalco
0.8164965809277261
claveria
claveria
1.0
coltongo
coltongo
1.0
cosmopolita
cosmopolita
1.0
cosmopolita ampl
cosmopolita
0.7071067811865475
cruz roja tepantongo u hab
tepantongo cruz roja
0.8660254037844388
del gas
del gas
0.9999999999999998
del gas ampl
del gas
0.816496580927726
del maestro
del maestro
0.9999999999999998
del recreo
del recreo
0.9999999999999998
ecologica novedades impacto u hab
ecologica novedades impacto
0.8660254037844388
el rosario a u hab
el rosario
0.816496580927726
el rosario b u hab
el rosario
0.816496580927726
el rosario c u hab
el rosario
0.816496580927726
euzkadi
euzkadi
1.0
exhacienda el rosario
el rosario
0.816496580927726
ferreria
ferreria
1.0
ferreria u hab
ferreria
0.7071067811865475
francisco villa u hab
francisco villa
0.816496580927726
fuentes de azcapotzalcoparques de azcapotzalc

In [29]:
result_df = marg_x_cols_df[marg_x_cols_df['CP']!='']
result_df.shape

(1634, 6)

In [30]:
result_df.isnull().sum()

,0
alcaldia,0
cve_col,0
colonia,0
pob_2010,0
grado_marginalidad,0
CP,0


In [31]:
result_com = result_df.merge(codigos_postales,on='CP',how='left')

In [34]:
result_com.drop(columns=['nombre_col','municipio'],inplace=True)

In [35]:
result_com.drop_duplicates(['colonia','CP']).count()

,0
alcaldia,1634
cve_col,1634
colonia,1634
pob_2010,1634
grado_marginalidad,1634
CP,1634
tipo_seccion,1634
tipo_col,1634


In [37]:
result_com.drop_duplicates(['colonia','CP']).to_csv('result_com_col_cp_dropdup.csv',index=False, header=True)

In [ ]:
codigos_postales[codigos_postales['nombre_col'].str.contains('UNION POPULAR EMILIANO ZAPATA',case=False)]

,ID,ENTIDAD,DISTRITO,MUNICIPIO,NOMBRE DEL MUNICIPIO,SECCION,tipo_seccion,tipo_col,nombre_col,CP,CONTROL
6945,827,9,16,10,ALVARO OBREGON,3481,URBANO(A),UNIDAD HABITACIONAL,UNION POPULAR EMILIANO ZAPATA,1400,865
